Downloads books from Internet Archive based on specific query: used to generate academic periodical/society collections

In [12]:
from internetarchive import get_item, download, search_items
import pandas as pd
import glob
import re
import os

In [4]:
#set date ranges
year_min=1800
year_max=1900

#set book limit per query
limit=5

In [5]:
#query to search
#deatiled documentation at https://archive.org/developers/internetarchive/internetarchive.html#internetarchive-search

#use parenteses and quotations for exact matching, just text for partial/fuzzy matching

search="title:('royal institution')"
#search="creator:charles darwin"

In [6]:
ia_books=[]
count=0
for i in search_items(search):
    #get metadata fields
    try:
        item = get_item(i["identifier"])
    except:
        continue
    meta=item.item_metadata['metadata']
    id=i["identifier"]
    try:
        date=int(meta["date"])
    except:
        date=0
    title=meta["title"]
    try:
        author=meta["creator"]
    except:
        author=""
        
    #check date ranges
    if date<year_max and date>year_min:
        count+=1
        #breaks if enough downloaded
        if count>limit:
            break
        try:
            download(id, verbose=False, glob_pattern='*txt', destdir="ia_articles/")
            
            ia_books.append([id, title, author, date])

        except:
            continue

In [7]:
#save records csv
ia=pd.DataFrame(ia_books)
ia.columns=["id", "title", "author", "date"]
ia.to_csv("output.csv")

In [16]:
try:
    os.mkdir("clean")
except:
    pass

In [17]:
#minor cleaning and file renaming
for id in ia["id"]:
    #read raw file
    path=glob.glob('ia_articles/'+id+'/*djvu.txt')
    #check if exists
    if path[0]:
        f = open(path[0], "r", encoding="utf-8")
    else:
        continue
    text=f.read()
    
    #too short tends to be erroneous ocr process
    if len(text)<1000:
        continue
    
    #standardize whitespace
    text = text.replace('\r', '').replace('\n', '')
    #remove numbers and special characters
    text=re.sub(r'[^a-zA-Z.,!? ]+', '', text)
    #lowercase
    text=text.lower()
    #write to new cleaned file
    out = open("clean/"+id+".txt", "w")
    out.write(text)
    f.close() 
    out.close()